# Publaynet BERT Classifier





## Environment Setup
Import key libraries and working envorinments. 

In [ ]:
# !pip install transformers==4.2.2

In [3]:
transformers.__version__

'4.2.2'

In [20]:
# Importing the libraries needed
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import seaborn as sns
import transformers
import json
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
#from transformers.configuration_bert import BertConfig
import logging
logging.basicConfig(level=logging.ERROR)

##Get Training and Validation Dataset

In [21]:
with open('funsd/funsd_train_list_dict.json', 'r') as fp:
    train_list_dict = json.load(fp)

with open('funsd/funsd_test_list_dict.json', 'r') as fp:
    eval_list_dict = json.load(fp)

In [22]:
train_list_dict.keys()

dict_keys(['92091873', '91939637', '87533049', '01073843', '92586242', '0012529284', '71341634', '0001477983', '91315069_91315070', '0060207528', '91161344_91161347', '71366499', '00922237', '91355841', '93380187', '01150773_01150774', '0060165115', '91914407', '92081358_1359', '660978', '0011974919', '81749056_9057', '00093726', '0000999294', '81186212', '0060077689', '87672097', '11875011', '00851772_1780', '0011859695', '81619511_9513', '0001456787', '81310636', '0060080406', '0011973451', '91104867', '00836244', '0001476912', '12052385', '0060255888', '71206427', '0012199830', '0001463448', '0000990274', '716552', '00920222', '0012529295', '80728670', '93213298', '71108371', '0001209043', '88057519', '01191071_1072', '00836816', '91361993', '92298125', '00040534', '0011838621', '0060029036', '0000989556', '0060024314', '0001129658', '0071032790', '00838511_00838525', '87682908', '0060214859', '00070353', '93351929_93351931', '0001438955', '81619486_9488', '00920294', '0000971160', 

In [23]:
eval_list_dict.keys()

dict_keys(['82837252', '85201976', '86263525', '82251504', '93106788', '82573104', '87528321', '85240939', '82562350', '82253245_3247', '87093315_87093318', '83443897', '87332450', '89856243', '83635935', '82250337_0338', '82200067_0069', '92380595', '86236474_6476', '82491256', '85540866', '86244113', '83823750', '83594639', '87528380', '86220490', '87086073', '87147607', '82252956_2958', '83641919_1921', '82253058_3059', '87125460', '83624198', '82504862', '86328049_8050', '82254765', '86079776_9777', '83553333_3334', '83573282', '87137840', '87428306', '86230203_0206', '87594142_87594144', '91814768_91814769', '86075409_5410', '83772145', '82253362_3364', '85629964', '82092117', '83996357'])

In [19]:
eval_list_dict['82837252']['objects']['0']

{'id': 7412,
 'box': [140, 186, 179, 200],
 'category': 'question',
 'text': 'DATE:',
 'relations': {'0': {'name': 'answer', 'id': 8473, 'object': 7421}},
 'gcn_bert_large': [0.057231076061725616,
  0.04367358982563019,
  0.04971188306808472,
  -0.00449754111468792,
  0.00840899720788002,
  0.020118363201618195,
  0.04725451394915581,
  0.0016711237840354443,
  -0.005014966242015362,
  0.0278470516204834,
  0.06539274752140045,
  0.020788487046957016,
  -0.005496850237250328,
  0.0006100722239352763,
  0.04117555916309357,
  0.04269300401210785,
  0.07025939226150513,
  0.05321888253092766,
  0.0472104549407959,
  0.048451147973537445,
  0.010146746411919594,
  -0.00772374402731657,
  -0.003857735078781843,
  0.0560426339507103,
  0.046066343784332275,
  0.010603458620607853,
  -0.006646316032856703,
  0.03853141516447067,
  -0.004647953435778618,
  0.05285662040114403,
  0.05691283941268921,
  0.059824779629707336,
  0.05756156146526337,
  0.050120726227760315,
  0.07251019775867462,


In [31]:
import os
training_list = []
for i in train_list_dict.keys():
    for x in train_list_dict[i]['objects'].keys():
      training_list.append(train_list_dict[i]['objects'][x])

In [35]:
# training_list

In [8]:
df_train = pd.read_pickle('funsd/funsd_bert_list_train.pkl')
df_test = pd.read_pickle('funsd/funsd_bert_list_test.pkl')

In [37]:
df_train.shape

(7411, 6)

In [38]:
# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cpu


In [39]:
df_train.head(5)

,id,box,category,text,relations,gcn_bert_large
0,0,"[105, 134, 158, 149]",question,Location,"{'0': {'name': 'answer', 'id': 8473, 'object':...","[-0.003128426382318139, 0.013897834345698357, ..."
1,1,"[425, 130, 475, 145]",question,Division,"{'0': {'name': 'answer', 'id': 8474, 'object':...","[-0.005620114505290985, 0.026954488828778267, ..."
2,2,"[432, 226, 502, 243]",header,Invitations:,"{'0': {'name': 'question', 'id': 8475, 'object...","[-0.005865470506250858, 0.031202778220176697, ..."
3,3,"[103, 254, 139, 269]",question,Mugs,"{'0': {'name': 'header', 'id': 8478, 'object':...","[-0.00615231366828084, 0.027706364169716835, -..."
4,4,"[105, 300, 157, 315]",question,Posters,"{'0': {'name': 'header', 'id': 8480, 'object':...","[-0.005726724863052368, 0.0267567727714777, -0..."


In [28]:
train_list_dict['92091873']['objects']['0']

dict_keys(['id', 'box', 'category', 'text', 'relations', 'gcn_bert_large'])

In [40]:
new_df = df_train[['text', 'label','near_visual_feature',	'gcn_near_char_density',	'gcn_near_char_number',
                   'level1_parse_emb','level2_parse_emb','gcn_near_token_density','density','visual_feature','gcn_bert_predicted']]



KeyError: "['label', 'near_visual_feature', 'gcn_near_char_density', 'gcn_near_char_number', 'level1_parse_emb', 'level2_parse_emb', 'gcn_near_token_density', 'density', 'visual_feature', 'gcn_bert_predicted'] not in index"

In [ ]:
new_df_test = df_test[['text', 'label','near_visual_feature',	'gcn_near_char_density',	'gcn_near_char_number',
                   'level1_parse_emb','level2_parse_emb','gcn_near_token_density','density','visual_feature','gcn_bert_predicted']]



## Data Preprocessing

In [ ]:
from transformers import BertTokenizer
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# Defining some key variables that will be used later on in the training
MAX_LEN = 100
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = TRAIN_BATCH_SIZE*2
# EPOCHS = 1
LEARNING_RATE = 2e-05
# Change the pre-trained bert model
#tokenizer = BertTokenizer.from_pretrained('roberta-base') #Cased 

In [ ]:
class SentimentData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.targets = self.data.label
        self.gcn_visual_feature = dataframe.near_visual_feature
        self.visual_feature = dataframe.visual_feature
        self.gcn_bert_base = dataframe.gcn_bert_predicted
        self.parsing1 = dataframe.level1_parse_emb
        self.parsing2 = dataframe.level2_parse_emb
        self.char_density = dataframe.gcn_near_char_density
        self.char_number = dataframe.gcn_near_char_number
        self.density = dataframe.density
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float),
            'density': torch.tensor(self.density[index],dtype=torch.float),
            'gcn_bert_base': torch.tensor(self.gcn_bert_base[index],dtype=torch.float),
            'char_density': torch.tensor(self.char_density[index],dtype=torch.float),
            'char_number': torch.tensor(self.char_number[index],dtype=torch.float),
            'visual_feature': torch.tensor(self.visual_feature[index],dtype=torch.float),
            'parsing1': torch.tensor(self.parsing1[index],dtype=torch.float),
            'parsing2': torch.tensor(self.parsing2[index],dtype=torch.float),
            'gcn_visual_feature': torch.tensor(self.gcn_visual_feature[index],dtype=torch.float),
        }

In [ ]:
train_size = 1
train_data=new_df.sample(frac=train_size,random_state=200)
#test_data=new_df.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)


print("FULL Dataset: {}".format(new_df.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(new_df_test.shape))

training_set = SentimentData(train_data, tokenizer, MAX_LEN)
#testing_set = SentimentData(test_data, tokenizer, MAX_LEN)
test_set = SentimentData(new_df_test,tokenizer,MAX_LEN)

In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
#testing_loader = DataLoader(testing_set, **test_params)
vali_loader = DataLoader(test_set, **test_params)

## Define the proposed classifiers

In [ ]:
class RobertaClass(torch.nn.Module):
    def __init__(self):
        super(RobertaClass, self).__init__()
        #bert-base-cased 768
        #bert-large-cased bert-large-uncased 1024
        #roberta-base-cased 768
        #biobert

        self.l1 = AutoModel.from_pretrained("bert-base-uncased")# BERT large
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.1)
        self.hidden_cls = torch.nn.Linear(768,768)
        self.hidden_parsing = torch.nn.Linear(768,768)
        self.hidden_den = torch.nn.Linear(768,768)
        self.hidden_vis = torch.nn.Linear(768,768)
        self.hidden_vis_pro = torch.nn.Linear(768,768)
        self.hidden_all = torch.nn.Linear(768*2,768*2)
        self.before_classifier = torch.nn.Linear(768*2,128)
\
        self.pooling = torch.nn.MaxPool2d((2,1), stride=None)
        self.classifier = torch.nn.Linear(128, 4)

    def forward(self, input_ids, attention_mask, token_type_ids, char_density,char_number,visual_feature,bert_cls,parsing1,parsing2,visual):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]

        # BERT 768 BERT / large 1024
        
        # set different hidden layer, number of hidden units, regularization methods including bn and dropout
        
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.Tanh()(pooler)
        pooler = self.dropout(pooler)

        pooler = torch.cat((pooler.unsqueeze(1),bert_cls.unsqueeze(1)),1)
        pooler = self.pooling(pooler).squeeze(1)
        pooler = self.hidden_cls(pooler)
        pooler = torch.nn.Tanh()(pooler)
        pooler = self.dropout(pooler)

        visual = self.hidden_vis_pro(visual)
        visual = torch.nn.Tanh()(visual)
        visual = self.dropout(visual)

        visual = torch.cat((visual.unsqueeze(1),visual_feature.unsqueeze(1)),1)
        visual = self.pooling(visual).squeeze(1)
        visual = self.hidden_vis(visual)
        visual = torch.nn.Tanh()(visual)
        visual = self.dropout(visual)

        pooler = torch.cat((pooler,visual),1)
        pooler = self.hidden_all(pooler)
        pooler = torch.nn.Tanh()(pooler)
        pooler = self.dropout(pooler)

        pooler = self.before_classifier(pooler)
        pooler = torch.nn.Tanh()(pooler)
        pooler = self.dropout(pooler)

        output = self.classifier(pooler)
        return output

In [ ]:
model = RobertaClass()
model.to(device)

## Training 

In [ ]:
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=1e-05) # change learning rate

In [ ]:
def calcuate_accuracy(preds, targets):
    n_correct = (preds==targets).sum().item()
    return n_correct

In [ ]:
# Defining the training function on the 80% of the dataset for tuning the distilbert model

def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    output = []
    model.train()
    for _,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)
        visual_feature = data['visual_feature'].to(device, dtype = torch.float)
        gcn_visual_feature = data['gcn_visual_feature'].to(device, dtype = torch.float)
        gcn_bert_base = data['gcn_bert_base'].to(device, dtype = torch.float)
        parsing1 = data['parsing1'].to(device, dtype = torch.float)
        parsing2 = data['parsing2'].to(device, dtype = torch.float)
        char_density = data['char_density'].to(device, dtype = torch.float)
        char_number = data['char_number'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids,char_density,char_number,gcn_visual_feature,gcn_bert_base,parsing1,parsing2,visual_feature)

        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accuracy(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        
        if _%5000==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples 

        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return

## Validation 

In [ ]:
def valid(model, testing_loader):
    model.eval()
    n_correct = 0; n_wrong = 0; total = 0; tr_loss=0; nb_tr_steps=0; nb_tr_examples=0
    output_list = []
    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            char_den = data['char_density'].to(device, dtype = torch.float)
            density = data['density'].to(device, dtype = torch.float)
            visual_feature = data['visual_feature'].to(device, dtype = torch.float)
            gcn_bert_base = data['gcn_bert_base'].to(device, dtype = torch.float)
            parsing1 = data['parsing1'].to(device, dtype = torch.float)
            parsing2 = data['parsing2'].to(device, dtype = torch.float)
            char_density = data['char_density'].to(device, dtype = torch.float)
            char_number = data['char_number'].to(device, dtype = torch.float)
            token_density = data['token_density'].to(device, dtype = torch.float)
            token_number = data['token_number'].to(device, dtype = torch.float)


            outputs = model(ids, mask, token_type_ids,visual_feature,char_density,char_number,token_density,token_number).squeeze()
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)
            output_list = output_list + list(big_idx)
            n_correct += calcuate_accuracy(big_idx, targets)

            nb_tr_steps += 1
            nb_tr_examples+=targets.size(0)
            
            
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")
    
    return epoch_accu,output_list


In [ ]:
acc,pre_list = valid(model, vali_loader)

## Testing

In [ ]:
class SentimentData_test(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.targets = self.data.label
        self.gcn_visual_feature = dataframe.near_visual_feature
        self.visual_feature = dataframe.visual_feature
        self.gcn_bert_base = dataframe.gcn_bert_predicted
        self.parsing1 = dataframe.level1_parse_emb
        self.parsing2 = dataframe.level2_parse_emb
        self.char_density = dataframe.gcn_near_char_density
        self.char_number = dataframe.gcn_near_char_number
        self.density = dataframe.density
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float),
            'density': torch.tensor(self.density[index],dtype=torch.float),
            'gcn_bert_base': torch.tensor(self.gcn_bert_base[index],dtype=torch.float),
            'char_density': torch.tensor(self.char_density[index],dtype=torch.float),
            'char_number': torch.tensor(self.char_number[index],dtype=torch.float),
            'visual_feature': torch.tensor(self.visual_feature[index],dtype=torch.float),
            'parsing1': torch.tensor(self.parsing1[index],dtype=torch.float),
            'parsing2': torch.tensor(self.parsing2[index],dtype=torch.float),
            'gcn_visual_feature': torch.tensor(self.gcn_visual_feature[index],dtype=torch.float),
        }

### load the test datasets

In [ ]:
new_df_true_test = new_df_test[['text', 'label','near_visual_feature',	'gcn_near_char_density',	'gcn_near_char_number',
                   'level1_parse_emb','level2_parse_emb','gcn_bert_predicted','visual_feature','gcn_near_token_density','density']]

In [ ]:
test = SentimentData_test(new_df_true_test,tokenizer, MAX_LEN)
testing_loader = DataLoader(test, **test_params)

In [ ]:
def test_label_generator(model, testing_loader):
    model.eval()
    n_correct = 0; n_wrong = 0; total = 0; tr_loss=0; nb_tr_steps=0; nb_tr_examples=0
    output_list = []
    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            visual_feature = data['visual_feature'].to(device, dtype = torch.float)
            gcn_visual_feature = data['gcn_visual_feature'].to(device, dtype = torch.float)
            gcn_bert_base = data['gcn_bert_base'].to(device, dtype = torch.float)
            parsing1 = data['parsing1'].to(device, dtype = torch.float)
            parsing2 = data['parsing2'].to(device, dtype = torch.float)
            char_density = data['char_density'].to(device, dtype = torch.float)
            char_number = data['char_number'].to(device, dtype = torch.float)

            outputs = model(ids, mask, token_type_ids,char_density,char_number,gcn_visual_feature,gcn_bert_base,parsing1,parsing2,visual_feature).squeeze()

            
            big_val, big_idx = torch.max(outputs.data, dim=1)
            output_list = output_list + list(big_idx)

            nb_tr_steps += 1
            
    return output_list


In [ ]:
EPOCHS = 4
for epoch in range(EPOCHS):
  train(epoch)
  output = test_label_generator(model, testing_loader)
  q = []
  for p in output:
    q.append(p.cpu().numpy().tolist())
  p = new_df_test['label'].tolist()
  from sklearn.metrics import classification_report, confusion_matrix
  report = classification_report(p,q, digits=4)
  print(report)  